In [ ]:
!pip install pandas
!pip install openpyxl
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Daten laden
data = pd.read_excel('SpotifyTopSongs_1000.xlsx')

# Ziele und Merkmale definieren
X = data.drop(columns=['popularity', 'snapshot_date', 'album_release_date'])  # Alle Spalten ausser 'popularity'
y = data['popularity']  # Zielvariable

# Datensatz in Train- und Test-Sätze aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X.describe()

,daily_rank,daily_movement,weekly_movement,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.00000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,25.475475,0.495495,1.513514,208566.714715,0.647178,0.618884,5.451451,-7.964455,0.534535,0.09398,0.282548,0.042679,0.176527,0.501089,120.492732,3.894895
std,14.424468,5.833506,12.201694,55585.318535,0.163959,0.187698,3.666371,6.234329,0.499056,0.08988,0.262412,0.166803,0.123270,0.238989,27.187782,0.374492
min,1.000000,-21.000000,-30.000000,104026.000000,0.142000,0.003320,0.000000,-46.113000,0.000000,0.02520,0.000132,0.000000,0.019100,0.000010,61.106000,1.000000
25%,13.000000,-2.000000,-5.000000,170925.500000,0.536000,0.521000,2.000000,-8.222500,0.000000,0.03715,0.064300,0.000000,0.098500,0.318000,100.018000,4.000000
50%,25.000000,0.000000,0.000000,197708.000000,0.682000,0.636000,6.000000,-6.422000,1.000000,0.05920,0.200000,0.000000,0.131000,0.508000,119.056000,4.000000
75%,38.000000,2.000000,4.000000,233246.000000,0.768000,0.758000,9.000000,-4.952500,1.000000,0.10900,0.436000,0.000195,0.208000,0.676500,138.008000,4.000000
max,50.000000,43.000000,49.000000,578571.000000,0.955000,0.981000,11.000000,-0.766000,1.000000,0.61700,0.996000,0.995000,0.858000,0.972000,235.512000,5.000000


In [9]:
from sklearn.ensemble import RandomForestRegressor

# Modell initialisieren
model = RandomForestRegressor(random_state=42)

# Modell trainieren
model.fit(X_train, y_train)

ValueError: could not convert string to float: '6WUGsDp56TjkMRrF5WsElJ'

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Vorhersagen für den Testdatensatz
y_pred = model.predict(X_test)

# Fehler metrischen
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # Wurzel des MSE
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

### Dokumentation

#### 1. Datensatz aufteilen
Der Datensatz wurde in einen Trainingsdatensatz (80%) und einen Testdatensatz (20%) aufgeteilt.

#### 2. Algorithmus auswählen
Als Algorithmus wurde der **Random Forest Regressor** gewählt, da er robust ist und komplexe Beziehungen zwischen den Merkmalen und dem Ziel erfassen kann.

#### 3. Modell evaluieren
- **MAE:** {mae:.2f}
- **MSE:** {mse:.2f}
- **RMSE:** {rmse:.2f}
- **R² Score:** {r2:.2f}

#### 4. Fazit
Das Modell erreicht einen R²-Score von {r2:.2f}, was darauf hinweist, dass es {r2*100:.2f}% der Varianz im Zielattribut erklärt. Der MAE beträgt {mae:.2f}.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Load your dataset (replace 'your_dataset.csv' with your actual file)
df = pd.read_excel("C:/Users/Cezar/Downloads/SpotifyTopSongs_1000.xlsx")

# Step 1: Drop irrelevant non-numerical columns
columns_to_drop = ['spotify_id', 'name', 'artists', 'album_name']
df.drop(columns=columns_to_drop, inplace=True)

In [2]:

# Step 2: Encode categorical columns
# Identify categorical columns
categorical_columns = ['country', 'is_explicit']

# Use OneHotEncoder for nominal categorical variables
encoder = OneHotEncoder(drop='first', sparse_output=False)  # Drop first category to avoid multicollinearity
encoded_features = encoder.fit_transform(df[categorical_columns])

# Create a DataFrame for encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_columns))

# Drop original categorical columns and concatenate encoded features
df = df.drop(columns=categorical_columns)
df = pd.concat([df, encoded_df], axis=1)

In [3]:
# Step 3: Handle date/time columns
# Convert date columns to numerical features (year, month, day)
date_columns = ['snapshot_date', 'album_release_date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])  # Convert to datetime format
    df[f'{col}_year'] = df[col].dt.year
    df[f'{col}_month'] = df[col].dt.month
    df[f'{col}_day'] = df[col].dt.day
    df.drop(columns=[col], inplace=True)  # Drop the original date column

In [4]:
# Step 4: Separate features (X) and target (y)
# Assume 'daily_movement' is the target variable
X = df.drop(columns=['popularity'])
y = df['popularity']

In [5]:
# Step 5: Handle missing values (if any)
X.fillna(X.mean(), inplace=True)  # Fill missing values with the mean
y.fillna(y.mean(), inplace=True)  # Fill missing values in the target variable

In [6]:

# Step 6: Scale numerical features (optional for tree-based models)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [7]:
# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Step 8: Train the RandomForestRegressor model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

# 4 - Evaluation

In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Step 9: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 10: Evaluate the model's performance
r2 = r2_score(y_test, y_pred)  # R² Score
mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
rmse = mean_squared_error(y_test, y_pred)  # Root Mean Squared Error (RMSE)

# Print evaluation metrics
print("Model Performance:")
print(f"R^2 Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")


Model Performance:
R^2 Score: 0.6611
Mean Absolute Error (MAE): 6.7795
Mean Squared Error (MSE): 81.3018
